<font size=8>
    <center>FIFA Crawler</center>
</font>
<br>

 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/Crawler/Team_Crawler_Fifa.ipynb#Librairies" data-toc-modified-id="Librairies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Librairies</a></span></li><li><span><a href="http://localhost:8888/notebooks/Crawler/Team_Crawler_Fifa.ipynb#Fifa-Players-Crawler" data-toc-modified-id="Fifa-Players-Crawler-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fifa Players Crawler</a></span></li><li><span><a href="http://localhost:8888/notebooks/Crawler/Team_Crawler_Fifa.ipynb#Data-Processing" data-toc-modified-id="Data-Processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Processing</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Crawler/Team_Crawler_Fifa.ipynb#Change-Lastname-&amp;-Add-Country-Trigramme" data-toc-modified-id="Change-Lastname-&amp;-Add-Country-Trigramme-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Change Lastname &amp; Add Country Trigramme</a></span></li><li><span><a href="http://localhost:8888/notebooks/Crawler/Team_Crawler_Fifa.ipynb#Manage-Duplicates-Lastname" data-toc-modified-id="Manage-Duplicates-Lastname-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Manage Duplicates Lastname</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/Crawler/Team_Crawler_Fifa.ipynb#Pictures-Saver" data-toc-modified-id="Pictures-Saver-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Pictures Saver</a></span></li></ul></div>

# Librairies

In [2]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
import time

# Fifa Players Crawler

In [4]:
def get_team_df(dict_correspondances, countries):
    player_name, player_country, player_position, player_age, player_description, player_link = [], [], [], [], [], []
    browser = webdriver.Firefox()
    browser.get("https://www.fifa.com/worldcup/teams/")
    time.sleep(1)
    
    for country in countries:
        element = browser.find_elements_by_xpath("//div[@class='fi-team-card__flag']")[dict_correspondances[country]]
        browser.execute_script("arguments[0].click();", element)
        time.sleep(1)

        for p in range(23):
            try:
                player = browser.find_elements_by_xpath("//div[@class='fi-p']/a[@class='fi-p--link']")[p]
                new_url = player.get_attribute('href')
                browser.get(new_url)
                time.sleep(1)
                player_name.append(browser.find_element_by_xpath("//div[@class='fi-p__name']").text)
                print(player_name[-1])
                player_country.append(country)
                player_position.append(browser.find_element_by_xpath("//div[@class='fi-p__role']").text)
                player_age.append(browser.find_element_by_xpath("//div[@class='fi-p__profile-number__number']").text)
                try:
                    player_description.append(browser.find_element_by_xpath("//div[@class='fi-p__profile-text']").text)
                except NoSuchElementException:
                    player_description.append("")
                my_img = browser.find_elements_by_tag_name('image')
                for image in my_img:
                    link = image.get_attribute('xlink:href')
                    player_link.append(link)
                browser.back()
                time.sleep(1)
            except IndexError:
                print()

        browser.get("https://www.fifa.com/worldcup/teams/")
        time.sleep(2)
    
    df = pd.DataFrame({"Name":player_name, "Country":player_country, "Position":player_position, "Age":player_age, 
                   "Description":player_description, "Link":player_link})
    df["Firstname"] = df["Name"].apply(lambda x: x.split()[0].capitalize())
    df["Lastname"] = df["Name"].apply(lambda x: "".join(x.split()[1:]).capitalize())
    return df

dict_correspondances = {"Argentina": 0, "Australia": 1, "Belgium": 2, "Brazil": 3,
                        "Colombia": 4, "Costa": 5, "Croatia": 6,
                        "Denmark": 7, "Egypt": 8, "England": 9,
                        "France": 10, "Germany": 11, "Iran": 13,
                        "Morocco": 17, "Peru": 20, "Poland": 21,
                        "Portugal": 22, "Russia": 23, "Arabia": 24,
                        "Senegal":25, "Serbia": 26, "Spain": 27, 
                        "Sweden": 28, "Switzerland": 29, "Tunisia": 30, 
                        "Uruguay": 31}

df = get_team_df(dict_correspondances, ["Argentina"])

NAHUEL GUZMAN
FRANCO ARMANI
WILFREDO CABALLERO
GABRIEL MERCADO
NICOLAS TAGLIAFICO
CRISTIAN ANSALDI
FEDERICO FAZIO
MARCOS ACUNA
JAVIER MASCHERANO
MARCOS ROJO
NICOLAS OTAMENDI
EDUARDO SALVIO
LUCAS BIGLIA
EVER BANEGA
ANGEL DI MARIA
MAXIMILIANO MEZA
ENZO PEREZ
GIOVANI LO CELSO
CRISTIAN PAVON
GONZALO HIGUAIN
LIONEL MESSI
SERGIO AGUERO
PAULO DYBALA


In [27]:
df.to_csv("df_full.csv", index=False)
df.shape

(543, 8)

# Data Processing
## Change Lastname & Add Country Trigramme

In [12]:
df = pd.read_csv("df_full.csv")
print(df.shape)
df.head()

(566, 8)


,Age,Country,Description,Link,Name,Position,Firstname,Lastname
0,32,Argentina,"Solidly built and a born competitor, Nahuel Gu...",https://api.fifa.com/api/v1/picture/players/20...,NAHUEL GUZMAN,Goalkeeper,Nahuel,Guzman
1,31,Argentina,Franco Armani owes his place at Russia 2018 to...,https://api.fifa.com/api/v1/picture/players/20...,FRANCO ARMANI,Goalkeeper,Franco,Armani
2,36,Argentina,"Willy Caballero is an experienced, unfussy goa...",https://api.fifa.com/api/v1/picture/players/20...,WILFREDO CABALLERO,Goalkeeper,Wilfredo,Caballero
3,31,Argentina,A right-back who is solid in defence and knows...,https://api.fifa.com/api/v1/picture/players/20...,GABRIEL MERCADO,Defender,Gabriel,Mercado
4,25,Argentina,"“A natural full-back,” remarked Jorge Sampaoli...",https://api.fifa.com/api/v1/picture/players/20...,NICOLAS TAGLIAFICO,Defender,Nicolas,Tagliafico


In [14]:
def get_lastname(text):
    text = text.split()
    if len(text) == 1:
        return text[0].capitalize()
    else:
        return ''.join(text[1:]).capitalize()
    
def get_trigramme(text):
    return dict_trig[text]

dict_trig = {"Argentina": 'ARG', "Australia": 'AUS', "Belgium": 'BEL', "Brazil": 'BRA', "Colombia": 'COL', 
             "Costa": 'CRC', "Croatia": 'HRV', "Denmark": 'DEN', "Egypt": 'EGY', "England": 'ENG',
             "France": 'FRA', "Germany": 'GER', "Iran": 'IRN', "Morocco": 'MAR', "Peru": 'PER', 
             "Poland": 'POL', "Portugal": 'POR', "Russia": 'RUS', "Arabia": 'KSA', "Serbia": 'SER', 
             "Spain": 'SPA', "Sweden": 'SWE', "Switzerland": 'SWI', "Tunisia": 'TUN','United_States':'USA',
             "Senegal":"SEN", "Uruguay":"URU"}

df["Lastname"] = df["Name"].apply(lambda x: get_lastname(x))
df["Trigramme"] = df["Country"].apply(lambda x: get_trigramme(x))

## Manage Duplicates Lastname

In [15]:
print(df["Lastname"].value_counts()[:16])

Silva         7
Rodriguez     4
Fernandes     4
Guzman        3
Hawsawi       3
Sanchez       3
Biglia        2
Amrabat       2
Higuain       2
Kalinic       2
Caballero     2
Ramos         2
Salvio        2
Locelso       2
Mascherano    2
Tagliafico    2
Name: Lastname, dtype: int64


In [16]:
df.loc[df.Name == "BERNARDO SILVA", 'Lastname'] = "Bernardo"
df.loc[df.Name == "ADRIEN SILVA", 'Lastname'] = "Adrien"
df.loc[df.Name == "ANDRE SILVA", 'Lastname'] = "Andre"
df.loc[df.Name == "BRUNO FERNANDES", 'Lastname'] = "Bruno"
df.loc[df.Name == "OMAR HAWSAWI", 'Lastname'] = "Omar"
df.loc[df.Name == "MOTAZ HAWSAWI", 'Lastname'] = "Motaz"
df.loc[df.Name == "ALEXEY MIRANCHUK", 'Lastname'] = "Alexey"
df.loc[df.Name == "NOUREDDINE AMRABAT", 'Lastname'] = "Noureddine"
df.loc[df.Name == "MATHIAS JORGENSEN", 'Lastname'] = "Mathias"
df.loc[df.Name == "JOSE CUADRADO", 'Lastname'] = "Jose"
df.loc[df.Name == "LOVRE KALINIC", 'Lastname'] = "Lovre"
df.loc[df.Name == "DAVINSON SANCHEZ", 'Lastname'] = "Davinson"
df.loc[df.Name == "THORGAN HAZARD", 'Lastname'] = "Thorgan"
df.loc[df.Name == "KHADIM NDIAYE", 'Lastname'] = "Khadim"
df.loc[df.Name == "ALFRED NDIAYE", 'Lastname'] = "Alfred"
df.loc[df.Name == "PAPE ALIOUNE NDIAYE", 'Lastname'] = "Alioune"
df.loc[df.Name == "GASTON SILVA", 'Lastname'] = "Gaston"
df.loc[df.Name == "SERGEJ MILINKOVIC-SAVIC", 'Lastname'] = "Savic"
df.loc[df.Name == "KEVIN DEBRUYNE", 'Lastname'] = "Bruyne"

In [17]:
df_worldcup = df.rename(columns={"Trigramme":"Code","Position":"Post"}).copy()
df_worldcup.to_csv("data_worldcup.csv", index=False)

# Pictures Saver

In [3]:
!ls

Team_Crawler_Fifa.ipynb data_worldcup.xlsx      df_full.csv
data_worldcup.csv       data_worldcup_old.csv


In [10]:
df_worldcup[df_worldcup.Country == "Serbia"]

,Age,Country,Description,Link,Name,Post,Firstname,Lastname,Code
410,34,Serbia,Vladimir Stojkovic is Serbia’s first-choice go...,https://api.fifa.com/api/v1/picture/players/20...,VLADIMIR STOJKOVIC,Goalkeeper,Vladimir,Stojkovic,SER
411,22,Serbia,Predrag Rajkovic came to prominence when he le...,https://api.fifa.com/api/v1/picture/players/20...,PREDRAG RAJKOVIC,Goalkeeper,Predrag,Rajkovic,SER
412,26,Serbia,Despite progressing through the youth ranks at...,https://api.fifa.com/api/v1/picture/players/20...,MARKO DMITROVIC,Goalkeeper,Marko,Dmitrovic,SER
413,34,Serbia,Antonio Rukavina is a full-back who prefers to...,https://api.fifa.com/api/v1/picture/players/20...,ANTONIO RUKAVINA,Defender,Antonio,Rukavina,SER
414,33,Serbia,Although Dusko Tosic has yet to feature for Se...,https://api.fifa.com/api/v1/picture/players/20...,DUSKO TOSIC,Defender,Dusko,Tosic,SER
415,25,Serbia,Central defender Uros Spajic was a surprise in...,https://api.fifa.com/api/v1/picture/players/20...,UROS SPAJIC,Defender,Uros,Spajic,SER
416,34,Serbia,Branislav Ivanovic is generally deployed at ce...,https://api.fifa.com/api/v1/picture/players/20...,BRANISLAV IVANOVIC,Defender,Branislav,Ivanovic,SER
417,32,Serbia,Aleksandar Kolarov was named as Serbia’s new c...,https://api.fifa.com/api/v1/picture/players/20...,ALEKSANDAR KOLAROV,Defender,Aleksandar,Kolarov,SER
418,22,Serbia,Milos Veljkovic is part of the Serbian ‘Golden...,https://api.fifa.com/api/v1/picture/players/20...,MILOS VELJKOVIC,Defender,Milos,Veljkovic,SER
419,27,Serbia,Full-back Milan Rodic was uncapped before the ...,https://api.fifa.com/api/v1/picture/players/20...,MILAN RODIC,Defender,Milan,Rodic,SER


In [2]:
df_worldcup = pd.read_csv("data_worldcup.csv")
df_worldcup.head()

,Age,Country,Description,Link,Name,Post,Firstname,Lastname,Code
0,32,Australia,"Solidly built and a born competitor, Nahuel Gu...",https://api.fifa.com/api/v1/picture/players/20...,NAHUEL GUZMAN,Goalkeeper,Nahuel,Guzman,AUS
1,31,Australia,Franco Armani owes his place at Russia 2018 to...,https://api.fifa.com/api/v1/picture/players/20...,FRANCO ARMANI,Goalkeeper,Franco,Armani,AUS
2,36,Australia,"Willy Caballero is an experienced, unfussy goa...",https://api.fifa.com/api/v1/picture/players/20...,WILFREDO CABALLERO,Goalkeeper,Wilfredo,Caballero,AUS
3,31,Australia,NaN,https://api.fifa.com/api/v1/picture/players/20...,GABRIEL MERCADO,Defender,Gabriel,Mercado,AUS
4,25,Australia,"“A natural full-back,” remarked Jorge Sampaoli...",https://api.fifa.com/api/v1/picture/players/20...,NICOLAS TAGLIAFICO,Defender,Nicolas,Tagliafico,AUS


In [18]:
def image_downloader(name, team, link):
    path = "../DjangoWorldCup/worldcup/static/worldcup/img/players/"
    with open(path + team.capitalize() + '/' + name.capitalize() + '.jpg', 'wb') as handle:
            response = requests.get(link, stream=True)

            if not response.ok:
                print(response)

            for block in response.iter_content(1024):
                if not block:
                    break

                handle.write(block)

for country in ["Argentina"]:
    for n, t, l in zip(df_worldcup[df_worldcup["Country"] == country]["Lastname"].tolist(), 
                       df_worldcup[df_worldcup["Country"] == country]["Country"].tolist(), 
                       df_worldcup[df_worldcup["Country"] == country]["Link"].tolist()):
        image_downloader(n, t, l)